<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Time Series Forecasting on Number of Passengers for an airline</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider a major airlines as our client. The client requires us to forecast the number of passengers who will fly their airline. The ARIMA model has been widely utilized in many fields for forecasting since it is recognized to be reliable, efficient, and capable of predicting short-term share market movements. Hence, we'll build an ARIMA model to forecast the demand(passenger traffic) in Airplanes.
<br>
<br>
ARIMA(Auto Regressive Integrated Moving Average) is a combination of 2 models AR(Auto Regressive) and MA(Moving Average). It has 3 hyperparameters - P(auto regressive lags), d(order of differentiation), Q(moving avg.) which respectively comes from the AR, I and MA components. The AR part is correlation between prev and current time periods. To smooth out the noise, the MA part is used. The I part binds together the AR and MA parts.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Data:</b></p>
<p style = 'font-size:16px;font-family:Arial'>The data for this demonstration resides on Vantage. Here we have a synthetic data for number of passengers travelling through the airline monthly.
</p>

<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Initiate a connection to Vantage</li>    
    <li>Explore the dataset</li>
    <li>Check for Stationarity using Dickey-Fuller Test</li>
    <li>Make series stationary using TD_DIFF (D)</li>
    <li>Check for autocorrelation of the time series (Q)</li>
    <li>Check for partial autocorrelation of the time series (P)</li>
    <li>Using ARIMA((AutoRegressive Integrated Moving Average) model to forecast number of passengers</li>
        <ul>
            <li>7.1 Estimation step using TD_ARIMAESTIMATE</li>
            <li>7.2 Extract residuals</li>
            <li>7.3 Create table PLOT_AR_FIT for plotting</li>
            <li>7.4 Validation step using TD_ARIMAVALIDATE</li>
            <li>7.5 Extract residuals</li>
            <li>7.6 Create table PLOT_ARMA_FIT for plotting</li>
            <li>7.7 Forecast step using TD_ARIMAFORECAST</li>
            <li>7.8 Create table PLOT_ARMA_FORECAST for plotting</li>
            <li>7.9 Visualize the results</li>
        </ul>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>This steps connects with Vantage. You might be prompted to enter the password.</p>

In [170]:
%connect local

Connected: 'local' connection activated for user 'demo_user'


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Explore the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>The data is classified in date/time and the passengers travelling per month. Time series deals with 2 columns, one is temporal i.e. <b>Date</b> in this case and another is the value to be forecasted i.e. <b>Passengers</b>.</p>

In [171]:
select top 5 * from airpassengers order by "Date";

,Date,Passengers,seriesID
1,1949-01-01 00:00:00.000000,112,ABC
2,1949-02-01 00:00:00.000000,118,ABC
3,1949-03-01 00:00:00.000000,132,ABC
4,1949-04-01 00:00:00.000000,129,ABC
5,1949-05-01 00:00:00.000000,121,ABC


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Please right click on the cell in the image column and click on "view image" to view the plot created by following piece of code.</b></p>

In [5]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS("Passengers"),CONTENT(REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('line'),
            LEGEND('upper left'),
            TITLE('Number of Passengers Travelling Monthly')
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<img id="fig1" src="images/fig1.png" alt="Number of Passengers Travelling Monthly" width="400" />

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Check for Stationarity using Dickey-Fuller Test</b>
<br>
<b style = font-size:16px;font-family:Arial>What is Stationarity?</b>
<br>
<p style = 'font-size:16px;font-family:Arial'>Before applying any statistical model on a Time Series, the series has to be staionary, which means that, over different time periods,
<br>
a) It should have constant mean.
<br>
b) It should have constant variance or standard deviation.
<br>
c) Auto-covariance should not depend on time.
</p>
<p style = 'font-size:16px;font-family:Arial'> We can visually examine if the mean and variance are constant over different periods of time or not. Alternatively, Dickey-Fuller test is widely used to check for stationarity in the time series data.
<br>

<p style = 'font-size:16px;font-family:Arial'>The result of the Dickey-Fuller Test:
<br>
• ACCEPT means the null hypothesis is accepted. The process
is non-stationary and unit roots are present.
<br>
• REJECT means the null hypothesis is rejected. The process is
stationary, and no unit roots are present.</p>

In [49]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
 SERIES_SPEC(TABLE_NAME(airpassengers),ROW_AXIS(TIMECODE("Date")),
 SERIES_ID("seriesID"), PAYLOAD(FIELDS("Passengers"),CONTENT(REAL))),
 FUNC_PARAMS(ALGORITHM('TREND'))
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,1,0,144,TREND ,-4.639196964346928,0.0008857695636920808,ACCEPT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column NULL_HYP, the series is not stationary as the NULL Hypothesis is accepted. Hence we need to make this series stationary.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Make series stationary using TD_DIFF</b>
<p style = 'font-size:16px;font-family:Arial'>TD_DIFF transforms a stationary, seasonal, or nonstationary time series into a differenced time series. We are going to create a new table which is differenced. We'll check if the newly created table is stationary or not(Using Dickey-Fuller test). If not, we'll again use TD_DIFF recursively till the series becomes stationary.</p>

In [40]:
drop table diff1_air;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'diff1_air' does not exist.

In [41]:
EXECUTE FUNCTION INTO VOLATILE ART(diff1_air)
TD_DIFF(
    SERIES_SPEC(
            TABLE_NAME(airpassengers),
            ROW_AXIS(TIMECODE("Date")),
            SERIES_ID("seriesID"),
            PAYLOAD(
                FIELDS("Passengers"),
                CONTENT(REAL)
            )
    ),
    FUNC_PARAMS(
          LAG(12),
          DIFFERENCES(5),
          SEASONAL_MULTIPLIER(0)
    )
);

Success: 84 rows affected

In [47]:
select * from diff1_air;

,seriesID,ROW_I,OUT_Passengers
1,ABC,0,-53
2,ABC,1,20
3,ABC,2,-222
4,ABC,3,-222
5,ABC,4,-297
6,ABC,5,59
7,ABC,6,-6
8,ABC,7,65
9,ABC,8,-22
10,ABC,9,10


<p style = 'font-size:16px;font-family:Arial'>In the above result, <b>ROW_I</b> is Row identifier of the ordered result sets and <b>OUT_Passengers</b> is Transformed magnitudes of differenced time series elements.

In [53]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
 SERIES_SPEC(TABLE_NAME(diff1_air),ROW_AXIS(SEQUENCE(ROW_I)),
 SERIES_ID("seriesID"), PAYLOAD(FIELDS("OUT_Passengers"),CONTENT(REAL))),
 FUNC_PARAMS(ALGORITHM('TREND'))
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,1,0,84,TREND ,-2.874317941271524,0.1709873849344384,REJECT 


<p style = 'font-size:16px;font-family:Arial'>Looking at the right most column NULL_HYP, the series is stationary as the NULL Hypothesis is rejected. Differencing the series once gave us stationary series. Hence the value of <b>d = 1</b>. This value will be used further in ARIMA_ESTIMATE.</p>
<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Check for autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>TD_ACF calculates the autocorrelation or autocovariance of a time series. The autocorrelation and autocovariance show how the time series covaries or correlates with itself when delayed by a lag in time or space. Here we check autocorrelation with a maximum lag of 12 time steps.</p>

In [66]:
drop table ACFDemo;

Success: 0 rows affected

In [67]:
EXECUTE FUNCTION INTO VOLATILE ART(ACFDemo)
TD_ACF(
    SERIES_SPEC(
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(1),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        FUNC_TYPE(0),   -- Use 0 for autocorrelation, or 1 for autocovariance
        DEMEAN(1),
        QSTAT(0),
        ALPHA(0.1)
    )
);

Success: 13 rows affected

In [54]:
select * from ACFDemo sample 5;

,seriesID,ROW_I,OUT_Passengers,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,ABC,1,0.9480473407524919,0.13707113557928935,0.8109762051732026,1.0851184763317812
2,ABC,3,0.8066811554965004,0.28525455433476354,0.5214266011617369,1.0919357098312639
3,ABC,4,0.752625417388308,0.32530417402650247,0.42732124336180555,1.0779295914148106
4,ABC,8,0.6556104843250867,0.4245345588715491,0.2310759254535376,1.080145043196636
5,ABC,10,0.7027199209090716,0.4618411236306323,0.24087879727843936,1.1645610445397039


In [68]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(ACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(OUT_Passengers, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'>On examination of above graph, we can say that the value of <b>Q = 2</b>.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Check for partial autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>The TD_PACF function provides insight as to whether the function being modeled is stationary or not. The partial auto correlations are used to measure the degree of correlation between time series sample points. Here we check partial autocorrelation with a maximum lag of 12 time steps.</p>

In [49]:
DROP TABLE PACFDemo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'PACFDemo' does not exist.

In [50]:
EXECUTE FUNCTION INTO VOLATILE ART(PACFDemo)
TD_PACF (
    SERIES_SPEC(
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL)
        )
    ) ,
    FUNC_PARAMS(
        MAXLAGS(12),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        ALGORITHM(LEVINSON_DURBIN),
        ALPHA(0.1)
    )
);

Success: 13 rows affected

In [51]:
select * from PACFDemo sample 5;

,seriesID,ROW_I,OUT_Passengers,CONF_OFF_Passengers,CONF_LOW_Passengers,CONF_HI_Passengers
1,ABC,2,-0.2294218741171843,0.13707113557928938,-0.3664930096964737,-0.09235073853789491
2,ABC,6,0.0077276025905067495,0.13707113557928938,-0.12934353298878262,0.14479873816979613
3,ABC,7,0.12559713017611632,0.13707113557928938,-0.01147400540317306,0.2626682657554057
4,ABC,10,0.16605125980436383,0.13707113557928938,0.028980124225074455,0.3031223953836532
5,ABC,12,-0.13543110227801888,0.13707113557928938,-0.27250223785730826,0.0016400333012704937


In [57]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(PACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(OUT_Passengers, CONF_OFF_Passengers),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1, ,0,...


<p style = 'font-size:16px;font-family:Arial'> We need to check, for which value in x-axis, graph line drops to 0 in y-axis for 1st time.
<br>
From PACF(at y=0), get P
    
<br>
Here from the above plot, <b>P = 2</b>.
</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Using ARIMA((AutoRegressive Integrated Moving Average) model to forecast number of passengers</b>
<p style = 'font-size:16px;font-family:Arial'>
You run the functions in the following order:
<br>
1. Run ARIMAESTIMATE function to get the coefficients for the ARIMA model.
<br>
2. [Optional] Run ARIMAVALIDATE function to validate the the "goodness of fit" of the ARIMA model, when
FIT_PERCENTAGE is not 100 in TD_ARIMAESTIMATE.
<br>
3. Run the ARIMAFORECAST function with input from step 1 or step 2 to forecast the future periods
beyond the last observed period.
</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.1 Estimation step using TD_ARIMAESTIMATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAESTIMATE function estimates the coefficients corresponding to an ARIMA model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the TD_ARIMAVALIDATE and TD_ARIMAFORECAST functions. This function is for
univariate series.</p>

<br>

<p style = 'font-size:16px;font-family:Arial'>In the below code, the previously estimated parameters P, d and Q need to be passed in MODEL_ORDER(P, d, Q).</p>

In [74]:
DROP TABLE ART_EST;

Success: 0 rows affected

In [75]:
-- trial
EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
TD_ARIMAESTIMATE(
    SERIES_SPEC(
        TABLE_NAME(airpassengers),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("Passengers"),
            CONTENT(REAL))),
     FUNC_PARAMS(
        NONSEASONAL(MODEL_ORDER(2, 1, 2)),
        SEASONAL(MODEL_ORDER(2, 1, 2), PERIOD(12)),
        CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
        RESIDUALS(1), ALGORITHM(MLE),  FIT_PERCENTAGE(80)
    )
);

Success: 158 rows affected

In [64]:
-- EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
-- TD_ARIMAESTIMATE(
--     SERIES_SPEC(
--         TABLE_NAME(ebay_clean),
--         ROW_AXIS(TIMECODE("Date")),
--         SERIES_ID(seriesID),
--         PAYLOAD(
--             FIELDS("High"),
--             CONTENT(REAL))),
--      FUNC_PARAMS(
--         NONSEASONAL(MODEL_ORDER(2, 0, 1)),
--         SEASONAL(MODEL_ORDER(2, 0, 2), PERIOD(1)),
--         LAGS(AR(2, 4), MA(3), SAR(1, 2), SMA(1, 3)),
--         FIXED(0.1, -1000, 0.01, 0.1, -1000, 0.2, -1000, -1000),
--         INIT(0, 0.1, 0, 0.1, 0.3, 0, 0.2, 100),
--         CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
--         RESIDUALS(1), ALGORITHM(MLE),  FIT_PERCENTAGE(80),  MAX_ITERATIONS(100)
--     )
-- );

Success: 0 rows affected

In [131]:
SELECT CAST(seriesID AS VARCHAR(8)), CAST(ROW_I AS INTEGER), CAST(COEFF_NAME AS  VARCHAR(8)), CAST(COEFF_VALUE AS FORMAT 'ZZ9.99999999'),
    CAST(STD_ERROR AS  FORMAT 'ZZ9.99999999'), CAST(ZSTAT_VALUE AS  FORMAT 'ZZ9.99999999'), CAST(ZSTAT_PROB AS  FORMAT 'ZZ9.99999999')
FROM ART_EST;

,seriesID,ROW_I,COEFF_NAME,COEFF_VALUE,STD_ERROR,ZSTAT_VALUE,ZSTAT_PROB
1,ABC,0,A R 1 ,0.266691119328648,0.31339099861030134,0.8509852564727801,0.39477754282424754
2,ABC,1,A R 2 ,0.5277666996460454,0.24646181360294325,2.1413731073824365,0.032243961835041766
3,ABC,2,M A 1 ,-0.5689173091349715,0.32577205554434774,-1.7463662074524502,0.08074733800790138
4,ABC,3,M A 2 ,-0.4572793571369058,0.33781781806457156,-1.3536271110764797,0.17585537297671763
5,ABC,4,S A R 1 ,1.3384295705079798,0.43357337231971227,3.0869736380421364,0.0020220552817092306
6,ABC,5,S A R 2 ,-0.383610782843414,0.5845301689342066,-0.6562719996862854,0.5116491301066508
7,ABC,6,S M A 1 ,-1.643028100527088,0.5345379413306793,-3.07373522717009,0.002113969590295322
8,ABC,7,S M A 2 ,0.7717804583415652,0.7324060907874356,1.053760295073184,0.29199267598096207


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.2 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table ouput of previous estimation step.
</p>

In [ ]:
DROP TABLE AR_RESIDUALS;

Success: 19 rows affected

In [ ]:
CREATE TABLE AR_RESIDUALS AS (
EXECUTE FUNCTION
TD_EXTRACT_RESULTS( ART_SPEC(TABLE_NAME(ART_EST), LAYER(ARTFITRESIDUALS) ) ) )  WITH DATA;

Success: 0 rows affected

In [ ]:
select * from AR_RESIDUALS;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,ABC,2,132,131.96619386707687,0.03380613292314261
2,ABC,4,121,120.98924614664233,0.010753853357676377
3,ABC,5,135,134.9781741449491,0.021825855050919333
4,ABC,6,148,147.9692071510109,0.030792848989106218
5,ABC,8,136,135.98742515601324,0.012574843986761718
6,ABC,9,119,119.00485618239662,-0.004856182396626345
7,ABC,10,104,104.01874670084834,-0.018746700848332676
8,ABC,11,118,118.07777557176726,-0.07777557176726192
9,ABC,12,115,115.42698906922152,-0.4269890692215148
10,ABC,13,126,121.85859249066377,4.141407509336234


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.3 Create table PLOT_AR_FIT for plotting</b></p>
</p>

In [132]:
DROP TABLE PLOT_AR_FIT;

Success: 17 rows affected

In [133]:
CREATE TABLE PLOT_AR_FIT (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [135]:
INSERT INTO PLOT_AR_FIT SELECT 'ABC', ROW_I, ACTUAL_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 
INSERT INTO PLOT_AR_FIT SELECT 'ESTIMATED', ROW_I, CALC_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 

Success: 0 rows affected

Success: 226 rows affected

In [136]:
select * from PLOT_AR_FIT sample 5;

,DatasetID,ROW_I,FIT_MAGNITUDE
1,ESTIMATED,50,215.5432026104707
2,ABC,9,119
3,ESTIMATED,71,224.46784980623104
4,ABC,88,318
5,ABC,92,355


In [139]:
-- EXECUTE FUNCTION
-- TD_Plot
-- (
--     SERIES_SPEC
--     (
--         TABLE_NAME(PLOT_AR_FIT),
--         ROW_AXIS(SEQUENCE(ROW_I)),
--         SERIES_ID(DataSetID),
--         ID_SEQUENCE('[{"DatasetID":"ABC"},{"DatasetID":"ESTIMATED"}]'),
--         PAYLOAD
--         (
--            FIELDS(FIT_MAGNITUDE),
--            CONTENT(REAL)
--         )
--     ),
--     FUNC_PARAMS
--     (
--         WIDTH(1920),
--         HEIGHT(1080),
--         TITLE('ARMA ESTIMATED: High'),
--         PLOTS[
--             (
--                 TITLE ('ARMA ESTIMATED: ORIGINAL and ESTIMATED SERIES'),
--                 GRID(FORMAT('-')),
--                 TYPE('line'),
--                 SERIES[
--                        (
--                         ID(1),
--                         FORMAT('r--')
--                        ),
--                        (
--                         ID(2),
--                         FORMAT('b-')
--                        )
--                      ],
--                 MARKER('o'),
--                 LEGEND('upper left'),
--                 XLABEL('X SeqNo'),
--                 YLABEL('Y Magnitude')
--             )
--         ]
--     )
-- );

,DatasetID,ROW_I,IMAGE
1,   ,0,...


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.4 Validation step using TD_ARIMAVALIDATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAVALIDATE function provides data scientists with a metrics collection for model selection and the produced residuals, such that a number of model validation tests can be performed. The TD_ARIMAVALIDATE function performs in-sample forecasting for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), and ARIMA models. It also supports the extended BoxJenkins seasonal ARIMA model formula.
</p>
<p style = 'font-size:16px;font-family:Arial'>We'll use the output of previous estimation step to validate the model. The testing train-validate split here for the dataset is 80:20. Hence 20% of data will be used for validation of the estimated model.</p>

In [147]:
DROP TABLE AR_VALIDATE;

Success: 0 rows affected

In [148]:
EXECUTE FUNCTION 
INTO VOLATILE ART(AR_VALIDATE)
TD_ARIMAVALIDATE(
	 ART_SPEC(TABLE_NAME(ART_EST)),
	 FUNC_PARAMS( FIT_METRICS(1),RESIDUALS(1)));

Success: 36 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.5 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table ouput of previous validation step.
</p>

In [149]:
DROP TABLE AR_VALIDATE_RESIDUALS;

Success: 19 rows affected

In [150]:
CREATE TABLE AR_VALIDATE_RESIDUALS AS (
EXECUTE FUNCTION 
TD_EXTRACT_RESULTS( ART_SPEC(TABLE_NAME(AR_VALIDATE), LAYER(ARTFITRESIDUALS) ) ) )  WITH DATA;

Success: 0 rows affected

In [152]:
select * from AR_VALIDATE_RESIDUALS sample 5;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,ABC,104,359,359.0064525421843,-0.006452542184339854
2,ABC,113,548,548.041948817516,-0.04194881751598693
3,ABC,120,391,397.2340888721565,-6.234088872156543
4,ABC,117,362,357.7224740159444,4.277525984055627
5,ABC,109,406,405.95812400221786,0.04187599778213774


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.6 Create table PLOT_ARMA_FIT for plotting</b></p>
</p>

In [155]:
DROP TABLE PLOT_ARMA_VALIDATE_FIT;

Success: 17 rows affected

In [157]:
CREATE TABLE PLOT_ARMA_VALIDATE_FIT (DatasetID VARCHAR(10), ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [158]:
INSERT INTO PLOT_ARMA_VALIDATE_FIT SELECT 'ABC', ROW_I, ACTUAL_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0; 
INSERT INTO PLOT_ARMA_VALIDATE_FIT SELECT 'PREDICTED', ROW_I, CALC_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0;

Success: 0 rows affected

Success: 58 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.7 Forecast step using TD_ARIMAFORECAST</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAFORECAST function is used to forecast a user-defined number of periods based on
models fitted from the TD_ARIMAESTIMATE function.</p>
<p style = 'font-size:16px;font-family:Arial'>Here in the following cell, we use the estimated and validated model to forecast number of passengers for next 6 periods i.e. next 6 months.</p>

In [ ]:
DROP TABLE ARMA_FORECAST;

Success: 0 rows affected

In [ ]:
EXECUTE FUNCTION INTO VOLATILE ART(ARMA_FORECAST)
TD_ARIMAFORECAST(
           ART_SPEC(TABLE_NAME(AR_VALIDATE)),
           FUNC_PARAMS(FORECAST_PERIODS(6)));

Success: 6 rows affected

In [ ]:
select * from ARMA_FORECAST;

,seriesID,ROW_I,FORECAST_VALUE,LO_80,HI_80,LO_95,HI_95
1,ABC,0,446.20704841325494,434.251227169459,458.16286965705086,427.9222037653006,464.4918930612093
2,ABC,1,420.19079129115704,405.2670573966333,435.1145251856808,397.36691744222514,443.01466514008894
3,ABC,2,456.02946271964674,438.49111607817446,473.56780936111903,429.20688506629625,482.85204037299724
4,ABC,3,493.89030023732687,474.76264441438235,513.0179560602714,464.6370847475444,523.1435157271093
5,ABC,4,508.8730509642807,488.39132473593014,529.3547771926312,477.54896418089396,540.1971377476674
6,ABC,5,580.0737769127627,558.6394956119717,601.5080582135537,547.2928833435408,612.8546704819846


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.8 Create table PLOT_ARMA_FORECAST for plotting</b></p>

In [160]:
DROP TABLE PLOT_ARMA_FORECAST;

Success: 17 rows affected

In [161]:
CREATE TABLE PLOT_ARMA_FORECAST (DatasetID VARCHAR(16), ROW_I BIGINT, FORECAST_MAGNITUDE FLOAT);

Success: 0 rows affected

In [162]:
INSERT INTO PLOT_ARMA_FORECAST   SELECT 'FORECASTED', ROW_I, FORECAST_VALUE FROM ARMA_FORECAST; 
INSERT INTO PLOT_ARMA_FORECAST   SELECT 'UPPER_BOUND', ROW_I, HI_80 FROM ARMA_FORECAST ; 
INSERT INTO PLOT_ARMA_FORECAST   SELECT 'LOWER_BOUND', ROW_I, LO_80 FROM ARMA_FORECAST ; 

Success: 0 rows affected

Success: 0 rows affected

Success: 18 rows affected

In [163]:
select * from PLOT_ARMA_FORECAST;

,DatasetID,ROW_I,FORECAST_MAGNITUDE
1,UPPER_BOUND,2,473.56780936111903
2,LOWER_BOUND,2,438.49111607817446
3,FORECASTED,2,456.02946271964674
4,UPPER_BOUND,4,529.3547771926312
5,LOWER_BOUND,4,488.39132473593014
6,FORECASTED,4,508.8730509642807
7,UPPER_BOUND,5,601.5080582135537
8,LOWER_BOUND,5,558.6394956119717
9,FORECASTED,5,580.0737769127627
10,UPPER_BOUND,3,513.0179560602714


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.9 Visualize the results</b></p>
<p style = 'font-size:16px;font-family:Arial'>The below cell results in a combined plot for estimation, validation and forecasting step. The forecast plot has a confidence band of 80%.</p>

In [168]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(PLOT_AR_FIT), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":"ABC"},{"DatasetID":"ESTIMATED"}]'),
                PAYLOAD(FIELDS(FIT_MAGNITUDE),CONTENT(REAL))),
    SERIES_SPEC(TABLE_NAME(PLOT_ARMA_VALIDATE_FIT),ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":"ABC"},{"DatasetID":"PREDICTED"}]'),
                PAYLOAD(FIELDS(FIT_MAGNITUDE), CONTENT(REAL))),
    SERIES_SPEC(TABLE_NAME(PLOT_ARMA_FORECAST), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":"FORECASTED"},{"DatasetID":"UPPER_BOUND"},{"DatasetID":"LOWER_BOUND"}]'),
                PAYLOAD(FIELDS(FORECAST_MAGNITUDE), CONTENT(REAL))),
    FUNC_PARAMS(LAYOUT(2,2), WIDTH(1920),HEIGHT(1080),
                TITLE('ARIMA ESTIMATE, VALIDATE, FORECAST'),
                PLOTS[(ID(1), CELL(1,1),
                       TITLE ('ARIMA ESTIMATE'),
                       GRID(FORMAT('-')),TYPE('line'),
                       SERIES[(ID(1),FORMAT('r--')),
                              (ID(2),FORMAT('b-')) ],
                       MARKER('o'), LEGEND('upper left'), XLABEL('X SeqNo'), YLABEL('#Passengers') ),
                      (ID(2), CELL(1,2),
                       TITLE ('ARIMA VALIDATE'),
                       GRID(FORMAT('-')), TYPE('line'),
                       SERIES[(ID(1),FORMAT('r--')),
                              (ID(2),FORMAT('b-')) ],
                       MARKER('o'), LEGEND('upper left'), XLABEL('X SeqNo'),YLABEL('#Passengers') ),
                      (ID(3),CELL(2,2),
                       TITLE ('ARIMA FORECAST'),
                       GRID(FORMAT('-')),TYPE('line'),
                       SERIES[(ID(1), FORMAT('r--')),
                              (ID(2),FORMAT('b-')),
                              (ID(3),FORMAT('b-'))],
                       MARKER('o'),LEGEND('upper left'),XLABEL('X SeqNo'),YLABEL('#Passengers') ) ] ));

,DatasetID,ROW_I,IMAGE
1,   ,0,...
